# Segmenting and Clustering Neighborhoods in Toronto

In [22]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [23]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print(res)

<Response [200]>


In [136]:
bs = BeautifulSoup(res.text, 'html.parser')
tableAsList = bs.select('.wikitable tr td')
n = 3
completeList = list()

In [149]:
for idx in range(0, len(tableAsList)):
    if tableAsList[idx].find('a') != None and tableAsList[idx].find('a').string != None:
        completeList.append(tableAsList[idx].find('a').string.replace('\n',''))
    elif tableAsList[idx] != None and tableAsList[idx].string != None:
        completeList.append(tableAsList[idx].string.replace('\n',''))
    else:
        completeList.append(tableAsList[idx].string)
finalList = [completeList[i * n:(i + 1) * n] for i in range((len(completeList) + n - 1) // n )] 

In [150]:
df = pd.DataFrame(finalList)
df.columns = ['PostalCode', 'Borough', 'Neighbourhood']
df = df[df.Borough != "Not assigned"]
df.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [184]:
df.loc[df['Neighbourhood'] == "Not assigned", 'Neighbourhood'] = df['Borough']
neighbourhoodDataFrame = df.groupby(['PostalCode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [185]:
neighbourhoodDataFrame.shape

(103, 3)

In [188]:
data = pd.read_csv("G:\\My Docs\\Courses\\IBM DS\\CapstoneProject\\Data-Driven-Neighborhoods-Classification\\Geospatial_Coordinates.csv") 
data.columns = ['PostalCode', 'Latitude', 'Longitude']
# neighbourhoodDataFrame.merge(data, on='PostalCode', how='left')
neighbourhoodDataFrame = pd.merge(neighbourhoodDataFrame, data, how='left',
        left_on='PostalCode', right_on='PostalCode')
neighbourhoodDataFrame.head()

,PostalCode,Borough,Neighbourhood,Latitude_x,Longitude_x,Latitude_y,Longitude_y,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern,Rouge,Malvern,Roug...",43.806686,-79.194353,43.806686,-79.194353,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union,Highland ...",43.784535,-79.160497,43.784535,-79.160497,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill,Guildwood,Morn...",43.763573,-79.188711,43.763573,-79.188711,43.763573,-79.188711
3,M1G,Scarborough,"Woburn,Woburn,Woburn,Woburn,Woburn,Woburn",43.770992,-79.216917,43.770992,-79.216917,43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,Cedarbrae,Cedarbrae,Cedarbrae,Cedarb...",43.773136,-79.239476,43.773136,-79.239476,43.773136,-79.239476
